In [5]:
class Prestataire():
    import notebook_importer
    output_dir    = "./Prestataires/"
    nb_d_instance =0
    def __init__(self, largeur=10, hauteur=10):
        import os
        from Position    import Position 
        from Station     import Station
        from Logger      import Logger
        from Reparateur  import Reparateur
        import math
        import numpy as np
        
        Prestataire.nb_d_instance += 1
        self.id             = "Prestataire_%s"%(Prestataire.nb_d_instance)
        self.logger         = Logger(os.path.join(Prestataire.output_dir, "{id}.csv".format(id=self.id)))
        self.largeur        = largeur
        self.hauteur        = hauteur
        self.reseau         = []
        self.velos          = []
        for lattitude in range(0, self.hauteur, 1):
            for longitude in range(0, self.largeur, 1):
                emplacement = Position(lattitude, longitude)
                new_station = Station(emplacement)
                self.velos.extend(new_station.velos)
                self.reseau.append(new_station)
        self.nb_reparateurs = 1 + math.ceil(max(self.hauteur, self.largeur)/4)
        self.reparateurs = []
        for lot_de_station_a_reparer in np.array_split(np.array(self.reseau), self.nb_reparateurs):
            reparateur = Reparateur(self, lot_de_station_a_reparer)
            self.reparateurs.append(reparateur)
            
        
            
            
            
        self.log_columns    = ["id", "largeur","hauteur"]
        self.logger.log(",".join(self.log_columns))
        self.logger.log(",".join([str(self.__dict__[col]) for col in self.log_columns]))
        
    def is_position_a_station(self, position):
        nb_ligne = round(position.longitude)
        nb_col   = round(position.lattitude)
        indice   =  (nb_ligne * self.largeur) + (nb_col )
        if nb_ligne<0 or nb_col<0:
            return False    

        closest_station  = self.reseau[int(indice)]
        return closest_station.position.is_same_position( position)

    def get_station_at_position(self, position):
        nb_ligne = round(position.longitude)
        nb_col   = round(position.lattitude)
        indice   =  (nb_ligne * self.largeur) + (nb_col )
        return self.reseau[int(indice)]

    def get_closest_station(self, position):
        """
        @Todo : améliorer pour les valeurs en dehors du réseau
        """
        from Position import Position
        nb_ligne = round(position.longitude)
        nb_col   = round(position.lattitude)
        indice   =  (nb_ligne * self.largeur) + (nb_col )
        if nb_ligne<0 or nb_col<0:
            closest_station  = self.reseau[0]    
        try:
            closest_station  = self.reseau[int(indice)]
            if Position.is_same_position(closest_station.position, position):
                closest_station = self.reseau[int(indice)+1]
        except Exception as e:
            closest_station  = self.reseau[-1]
        return closest_station


    def filtrer_le_reseau(self, function_de_test):
        """
        Renvoie les stations qui passe le test.

        Parameters:
            function de test : lambda function
        Return:
            list

        """
        return list(filter(function_de_test, self.reseau))
    def nb_station_passant_le_test(self, function):
        r = self.filtrer_le_reseau(function)
        return len(r)

    def nombre_de_stations(self):
        return len(self.reseau)

    def nombre_de_velos(self):
        return sum(map(lambda x: x.nb_de_velos(),self.reseau))

    def nombre_de_stations_pleine(self):
        test  = lambda station: station.nb_bornes == len(station.velos)
        return self.nb_station_passant_le_test(test)

    def nombre_de_stations_avec_velo(self):
        test = lambda x: x.has_velo()
        return self.nb_station_passant_le_test(test)

    def nombre_de_stations_vide(self):
        test  = lambda station: not station.has_velo()
        return self.nb_station_passant_le_test(test)

    def __str__(self):
        return """
                Nb de stations           : %s 
                Nb de vélos              : %s 
                Nb de stations avec vélo : %s 
                Nb de stations pleines   : %s
                Nb de stations vides     : %s
        """%(self.nombre_de_stations(),
             self.nombre_de_velos(),
             self.nombre_de_stations_avec_velo(),
             self.nombre_de_stations_pleine(),
             self.nombre_de_stations_vide())

    def nombre_de_velos_sous_performants(self, seuil=0.5):
        r = map(lambda station: station.nb_velo_under_perf(seuil), self.reseau)
        return sum(list(r))
    def get_topology(self):
        """
        Return a Datframe of longitude/lattitude
        """
        import pandas as pd
        infos = { "longitudes" : [x.position.longitude for x in self.reseau],
                  "lattitude"  : [x.position.lattitude for x in self.reseau]}

        return pd.DataFrame(infos)

    def get_velos_data(self):
        import pandas as pd
        return pd.concat([velo.flush_data() for velo in self.velos])




In [6]:
p = Prestataire(1,1)